In [ ]:
!pip install datasets
!pip install datasets transformers==4.28.0
!pip install transformers datasets
!pip install transformers[torch]
!pip install accelerate -U
!pip install evaluate



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import transformers
import evaluate
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import f1_score
from datasets import load_dataset
from datasets import Dataset

from gensim.utils import simple_preprocess
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix

import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import  DataLoader
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler, TrainingArguments, Trainer

import re

from transformers import get_linear_schedule_with_warmup, AutoTokenizer, AutoModel, logging

import warnings
warnings.filterwarnings("ignore")

logging.set_verbosity_error()

In [ ]:
def seed_everything(seed_value):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

seed_everything(86)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 15
N_SPLITS = 4

In [ ]:
# pip install sacremoses

In [ ]:
# # model_name = "vinai/phobert-large"
model_name = "nam7197/vi-nli-xlm-roberta-base"
# #model_name = "vinai/phobert-base-v2"
#model_name = 'Fsoft-AIC/videberta-base'
# model_name1 = 'xlm-roberta-base'
# model_name = 'symanto/xlm-roberta-base-snli-mnli-anli-xnli'

# tokenizer1 = AutoTokenizer.from_pretrained("xlm-roberta-base")
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base' )

(…)lm-roberta-base/resolve/main/config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

(…)ase/resolve/main/sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

(…)roberta-base/resolve/main/tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def get_data(path):
    df = pd.read_json(path)
    df = df.transpose()

    # df.drop(columns=['context','domain'], inplace=True)
    df = df.reset_index(drop=True)
    return df


data = get_data('/content/drive/MyDrive/data/data_train.json')

In [ ]:
data

,claim,verdict,evidence
0,"Ngoài việc không giới hạn mức lương, công ty c...",SUPPORTED,công ty cũng có chế độ đãi ngộ tốt có lương th...
1,"Thành lập năm 2016, phát triển phần mềm, tổ ch...",SUPPORTED,bên cạnh đó faba thường xuyên tổ chức những dự...
2,"Nhằm phát triển đội ngũ, FABA thường tổ chức n...",SUPPORTED,bên cạnh đó faba thường xuyên tổ chức những dự...
3,Sở dĩ vị trí kỹ sư phần mềm có mức lương cao n...,SUPPORTED,không riêng kỹ sư phần mềm đối với bất kỳ lĩnh...
4,"Theo Adeco Việt Nam, mức lương cao nhất đối vớ...",SUPPORTED,theo adeco việt nam mức lương cao nhất đối với...
...,...,...,...
37962,Tỉ lệ tử vong do ung thư dạ dày hiện đang cao ...,NEI,theo thống kê của tổ chức ung thư thế giới (gl...
37963,Nam giới vẫn có khả năng có con một cách tự nh...,NEI,nam giới có khả năng sản xuất tinh trùng và du...
37964,Nữ giới mang thai sau 35 tuổi làm con tăng khả...,NEI,ở nữ giới tuổi tác không chỉ gây khó khăn tron...
37965,Nam giới nên có chế độ ăn hợp lí để duy trì ch...,NEI,nam giới có khả năng sản xuất tinh trùng và du...


In [ ]:
data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
!nvidia-smi

Tue Nov  7 04:57:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      2MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:

import plotly.express as px

In [ ]:
labels, frequencies = np.unique(data.verdict.values, return_counts = True)

fig = px.pie(values=frequencies,
             names=labels,
             title='Languages distribution',
             color_discrete_sequence=px.colors.sequential.Plotly3)

fig.show()

In [ ]:
import pandas as pd

# Assuming 'data' is a DataFrame with a 'verdict' column

label_mapping = {'SUPPORTED': 0, 'REFUTED': 1, 'NEI': 2}

def label_to_correct(label):
    if label is not None:
        return label_mapping.get(label.strip(), -1)
    else:
        return label_mapping.get(label, -1)
data['verdict'] = data['verdict'].apply(label_to_correct)


In [ ]:
# for i,key in data.iterrows():
#   if   key['evidence'] == "" :
#     print(i)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_labels = np.unique(data['verdict'])
class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=data['verdict'])
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

In [ ]:
class_weights

tensor([0.9898, 1.0046, 1.0058], device='cuda:0')

In [ ]:
import string

punctuations=list(string.punctuation)

In [ ]:
# punctuations

In [ ]:
def normalizer(text):

  def remove_meaningless_token(s):
    return re.sub(r'\b(jeez|yeah|uhhuh|uh|oh|huh|ừ|à)\b',' ',s)
  def uncased(s):
    return s.lower()

  def fix_whitespaces(s):
    return re.sub(r'\s+',' ',s)

  def remove_punctuations(s):
    chs=[]
    for ch in s:
      if ch not in punctuations:
        chs.append(ch)
      else:
        chs.append(" ")
    return "".join(chs)

  return fix_whitespaces(remove_meaningless_token(uncased(remove_punctuations(text)))).strip()

In [ ]:
data['claim']=data['claim'].apply(normalizer)
data['evidence']=data['evidence'].apply(normalizer)

In [ ]:
data

,claim,verdict,evidence
0,chú chó norman chỉ có thể trượt ván,1,chú chó này cũng có thể đạp xe đạp có bánh phụ...
1,ông huấn là bí thư chi bộ trường tiểu học ngư ...,1,hiện ông tuấn là bí thư còn ông huấn là phó bí...
2,cách di chuyển viên bi sẽ do người đeo dùng lư...,0,người đeo sẽ dùng lưỡi để di chuyển viên bi
3,năm 2021 lượng dầu nhập khẩu từ nga của trung ...,0,trung quốc đã tăng nhập khẩu dầu nga lên mức t...
4,dow chemical và monsanto là hai công ty sản xu...,2,do đó tòa bác đơn kiện của bà trần tố nga đối ...
...,...,...,...
37962,vài ngày gần đây những chiến đấu cơ hiện đại c...,1,vài tháng gần đây những chiến đấu cơ hiện đại ...
37963,một người dùng weibo bình luận thể hiện sự chê...,1,video về hoàng thu hút 56 triệu lượt xem trên ...
37964,để nối được với cồn đâu xã hải dương thì cần l...,0,các trụ cầu bắc qua đầm bầu hai để nối với cồn...
37965,tỏi rừng a hay còn được biết đến là tỏi rừng p...,0,loài mới được đặt tên là tỏi rừng phong điền a...


In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
# smote = SMOTE(sampling_strategy='auto')  # Có thể điều chỉnh sampling_strategy nếu cần
# X_resampled, y_resampled = smote.fit_resample(X, y)

In [ ]:
k = 5  # Số fold (tùy chọn)
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
data['kfold'] = np.nan
for fold, (_, val_) in enumerate(skf.split(X=data[['claim','evidence']], y=data['verdict'])):
    data.loc[val_, "kfold"] = fold


In [ ]:
data

,claim,verdict,evidence,kfold
0,chú chó norman chỉ có thể trượt ván,1,chú chó này cũng có thể đạp xe đạp có bánh phụ...,2.0
1,ông huấn là bí thư chi bộ trường tiểu học ngư ...,1,hiện ông tuấn là bí thư còn ông huấn là phó bí...,0.0
2,cách di chuyển viên bi sẽ do người đeo dùng lư...,0,người đeo sẽ dùng lưỡi để di chuyển viên bi,0.0
3,năm 2021 lượng dầu nhập khẩu từ nga của trung ...,0,trung quốc đã tăng nhập khẩu dầu nga lên mức t...,3.0
4,dow chemical và monsanto là hai công ty sản xu...,2,do đó tòa bác đơn kiện của bà trần tố nga đối ...,0.0
...,...,...,...,...
37962,vài ngày gần đây những chiến đấu cơ hiện đại c...,1,vài tháng gần đây những chiến đấu cơ hiện đại ...,0.0
37963,một người dùng weibo bình luận thể hiện sự chê...,1,video về hoàng thu hút 56 triệu lượt xem trên ...,2.0
37964,để nối được với cồn đâu xã hải dương thì cần l...,0,các trụ cầu bắc qua đầm bầu hai để nối với cồn...,3.0
37965,tỏi rừng a hay còn được biết đến là tỏi rừng p...,0,loài mới được đặt tên là tỏi rừng phong điền a...,4.0


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df,tokenizer, max_len=128):
        self.df = df
        self.max_len = max_len
        self.label_encoder = LabelEncoder()
        self.tokenizer = tokenizer
        self.df['verdict'] = self.label_encoder.fit_transform(self.df['verdict'])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        evidence = self.df.iloc[idx]['evidence']
        claim = self.df.iloc[idx]['claim']
        label = self.df.iloc[idx]['verdict']

        # Encode evidence and claim using PhoBERT tokenizer
        encoding = self.tokenizer.encode_plus(
            evidence,
            claim,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_token_type_ids=False,
            return_tensors='pt',
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_masks': attention_mask,
            'targets': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
def prepare_loaders(df, fold):
    df_train = df[df["kfold"] != fold].reset_index(drop=True)
    df_eval = df[df["kfold"] == fold].reset_index(drop=True)

    train_dataset = CustomDataset(df_train, tokenizer, max_len=120)
    valid_dataset = CustomDataset(df_eval, tokenizer, max_len=120)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    eval_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True, num_workers=2)

    return train_loader, eval_loader

In [ ]:
def train(model, criterion, optimizer, train_loader):
    model.train()
    losses = []
    correct = 0

    for data in train_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_masks'].to(device)
        targets = data['targets'].to(device)

        optimizer.zero_grad()
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        loss = criterion(outputs, targets)
        _, pred = torch.max(outputs, dim=1)

        correct += torch.sum(pred == targets)
        losses.append(loss.item())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        lr_scheduler.step()

    print(f'Train Accuracy: {correct.double()/len(train_loader.dataset)} Loss: {np.mean(losses)}')

In [ ]:

from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    return accuracy, precision, recall, f1

In [ ]:
def eval1( valid_loader):

    model.eval()
    losses = []
    correct = 0
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        data_loader = valid_loader
        for data in data_loader:
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_masks'].to(device)
            targets = data['targets'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, pred = torch.max(outputs, dim=1)

            loss = criterion(outputs, targets)
            correct += torch.sum(pred == targets)
            losses.append(loss.item())

            all_predictions.extend(pred.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
        accuracy, precision, recall, f1 = calculate_metrics(all_targets, all_predictions)
        accuracy = correct.double() / len(valid_loader.dataset)
        f1 = f1_score(all_targets, all_predictions, average='weighted')  # Calculate F1 score

        print(f'Test Accuracy: {accuracy} Loss: {np.mean(losses)} F1 Score: {f1} Precision: {precision} Recall: {recall}')
        return accuracy, f1, np.mean(losses)

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel

class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.drop1 = nn.Dropout(p=0.3)
        self.fc1 = nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size // 2)
        self.drop2 = nn.Dropout(p=0.3)  # Add a second Dropout layer
        self.fc2 = nn.Linear(self.bert.config.hidden_size // 2, n_classes)
        self.leakyrelu = nn.LeakyReLU(0.1)
        self.batch_norm = nn.BatchNorm1d(self.bert.config.hidden_size // 2)
        self.softmax = torch.nn.Softmax(dim=1)
        nn.init.normal_(self.fc1.weight, std=0.1)
        nn.init.normal_(self.fc1.bias, 0.1)
        nn.init.normal_(self.fc2.weight, std=0.1)
        nn.init.normal_(self.fc2.bias, 0.1)

    def forward(self, input_ids, attention_mask):
        last_hidden_state, output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False
        )

        x = self.drop1(output)
        x = self.fc1(x)
        x = self.leakyrelu(x)
        x = self.batch_norm(x)
        x = self.drop2(x)  # Apply a second Dropout layer
        x = self.fc2(x)
        x = self.softmax(x)
        return x


In [ ]:
model = SentimentClassifier(n_classes=3).to(device)

(…)lm-roberta-base/resolve/main/config.json:   0%|          | 0.00/874 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [ ]:
model

SentimentClassifier(
  (bert): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
       

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/model')

In [ ]:
os.getcwd()

'/content/drive/MyDrive/model'

In [ ]:
from torch.nn.modules import loss
for fold in range(skf.n_splits):

    print(f'-----------Fold: {fold+1} ------------------')
    train_loader, valid_loader = prepare_loaders(data, fold=fold)
    criterion = nn.CrossEntropyLoss(weight = class_weights)

    # Recommendation by BERT: lr: 5e-5, 2e-5, 3e-5
    # Batchsize: 16, 32

    optimizer = AdamW(model.parameters(), lr=1e-5,weight_decay=0.02)

    lr_scheduler = get_linear_schedule_with_warmup(
                optimizer,
                num_warmup_steps=0,
                num_training_steps=len(train_loader)*EPOCHS
            )
    best_f1 = 0
    loss1 = 5
    for epoch in range(EPOCHS):
        print(f'Epoch {epoch+1}/{EPOCHS}')
        print('-'*30)

        train(model, criterion, optimizer, train_loader)
        val_acc,valf1,lss = eval1(valid_loader)
        if valf1 > best_f1 and lss < loss1 :
            torch.save(model.state_dict(), f'phobert_fold{fold+1}.pth')
            best_f1 = valf1
            loss1 = lss



-----------Fold: 1 ------------------
Epoch 1/15
------------------------------
Train Accuracy: 0.7716722088697198 Loss: 0.7757959429213875
Test Accuracy: 0.7894390308138004 Loss: 0.7572164072209046 F1 Score: 0.7905368578688489 Precision: 0.8057670791577188 Recall: 0.7894390308138004
Epoch 2/15
------------------------------
Train Accuracy: 0.8146050768774898 Loss: 0.7347766236255043
Test Accuracy: 0.8058993942586252 Loss: 0.7412527504087496 F1 Score: 0.8052382504928759 Precision: 0.8096233453423829 Recall: 0.8058993942586252
Epoch 3/15
------------------------------
Train Accuracy: 0.8311329141013399 Loss: 0.7184862527094389
Test Accuracy: 0.8172241243086648 Loss: 0.7323871292486912 F1 Score: 0.8173623810097136 Precision: 0.8188948549258664 Recall: 0.8172241243086648
Epoch 4/15
------------------------------
Train Accuracy: 0.8424587627168867 Loss: 0.7070881847958815
Test Accuracy: 0.8330260732156967 Loss: 0.7156449709118915 F1 Score: 0.8330307279748759 Precision: 0.833428372641561 Re

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Custom TB Handler failed, unregistering
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Test Accuracy: 0.9525879099170289 Loss: 0.5984861833207747 F1 Score: 0.9526248894108488 Precision: 0.953032522341456 Recall: 0.9525879099170288
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-38-27dcef946687>", line 27, in <cell line: 2>
    torch.save(model.state_dict(), f'phobert_fold{fold+1}.pth')
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 618, in save
    with _open_zipfile_writer(f) as opened_zipfile:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 492, in _open_zipfile_writer
    return container(name_or_buffer)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 463, in __init__
    super().__init__(torch._C.PyTorchFileWriter(self.name))
RuntimeError: File phobert_fold5.pth cannot be opened.

During handling of the above e

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-38-27dcef946687>", line 27, in <cell line: 2>
    torch.save(model.state_dict(), f'phobert_fold{fold+1}.pth')
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 618, in save
    with _open_zipfile_writer(f) as opened_zipfile:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 492, in _open_zipfile_writer
    return container(name_or_buffer)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 463, in __init__
    super().__init__(torch._C.PyTorchFileWriter(self.name))
RuntimeError: File phobert_fold5.pth cannot be opened.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interacti

In [ ]:
# từ đây xuống là inference

In [ ]:
# model.to('cpu')
# def predict(claim, context):
#   # Tokenize claim và context
#   model.load_state_dict(torch.load('/content/drive/MyDrive/check point/phobert_fold2.pth',map_location=torch.device('cpu')))
#   inputs = tokenizer(
#       claim,
#       context,
#       padding=True,
#       truncation=True,
#       return_tensors="pt"
#   )
#   input_ids = inputs["input_ids"].to('cpu')
#   attention_mask = inputs["attention_mask"].to('cpu')


#     # Dự đoán
#   with torch.no_grad():

#       outputs = model(input_ids, attention_mask=attention_mask)
#       probabilities = torch.softmax(outputs, dim=1)
#       print(probabilities)

#   # Lấy các lớp và xác suất dự đoán tương ứng
#   labels = ['neutral', 'support']
#   predicted_label = labels[torch.argmax(probabilities, axis=1).item()]
#   predicted_probabilities = probabilities.tolist()[0]

#   return predicted_label, predicted_probabilities


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# model.load_state_dict(torch.load('/content/phobert_fold4.pth',map_location=torch.device('cpu')))

In [ ]:
# def get_data(path):
#     df = pd.read_json(path)
#     df = df.transpose()
#     df.columns = ['verdict','claim', 'evidence']
#     df = df.reset_index(drop=True)
#     return df


# data = get_data('/content/label_nei_full.json')

In [ ]:
# print(data['claim'][i])
# print(data['evidence'][i])

In [ ]:
# claim = data['claim'][i]
# context = data['evidence'][i]

In [ ]:
# label,predit_prob = predict(claim,context)

In [ ]:
# label

In [ ]:
# predit_prob